# Create a Dataset from scratch and run it

We want to create a project that contains all the jobs to calculate the Silicon band structure with PBE. To set up the jobs we will use the schemes in the `Schemes` class.

In [1]:
# import packages
from pynter.data.datasets import Dataset
from pynter.tools.materials_project import MPDatabase
from pynter.vasp.schemes import Schemes

In [ ]:
# define the project path - customize with your path
import os
homedir = os.getenv("HOME")
path = os.path.join(homedir,'pynter-tutorials-files/Si-BS-dataset')

# get structure of FCC Si from Materials Project
structure = MPDatabase(mp_id='mp-149').get_structure()

# Adapt job settings for a smaller job
job_settings = {'nodes':1,'timelimit':'00:30:00'}

cs = Schemes(path=path,structure=structure,job_settings=job_settings,name='Si-BS')
jobs = cs.pbe_electronic_structure()

# create Dataset
ds = Dataset(jobs)

In [4]:
# visualize job table
ds.jobs_table()

,formula,group,nodes,is_converged
job_name,,,,
Si-BS_PBE-el-str_1,Si2,1-PBE-relax,,None
Si-BS_PBE-el-str_2,Si2,2-PBE-DOS,,None
Si-BS_PBE-el-str_3,Si2,3-PBE-BS,,None


In [6]:
# get first step job
job_first_step = ds.select_jobs(groups=['1-PBE-relax'])
print(job_first_step)

Job "Si-BS_PBE-el-str_1" of group "1-PBE-relax"


In [7]:
# check job inputs
job_first_step.inputs

{'INCAR': {'IBRION': 2,
  'NSW': 100,
  'ISIF': 3,
  'EDIFFG': -0.05,
  'ISPIN': 1,
  'LWAVE': '.TRUE.',
  'LCHARG': '.TRUE.',
  'LORBIT': 10,
  'LVTOT': '.TRUE.',
  'ENCUT': 550,
  'EDIFF': 1e-06,
  'ISMEAR': 0,
  'SIGMA': 0.05,
  'NELM': 200,
  'ALGO': 'Normal',
  'AMIX': 0.2,
  'LREAL': '.FALSE.',
  'SYSTEM': 'Si',
  '#### Default PBE: system': 'Si',
  'ISYM': 2},
 'KPOINTS': pymatgen v2020.7.18 with grid density = 1000 / number of atoms
 0
 Gamma
 8 8 8,
 'POSCAR': Si2
 1.0
 3.325489 0.000000 1.919972
 1.108496 3.135301 1.919972
 0.000000 0.000000 3.839943
 Si
 2
 direct
 0.875000 0.875000 0.875000 Si
 0.125000 0.125000 0.125000 Si,
 'POTCAR': [<pymatgen.io.vasp.inputs.PotcarSingle at 0x7f777b024c10>]}

In [6]:
# write input to files
ds.write_jobs_input()

In [7]:
# sync Dataset folder with HPC to transfer all the files before starting calculations
# In order for the automations to work all files need to be present in HPC
ds.sync_dataset_to_hpc()

sending incremental file list
1-PBE-SCF/
1-PBE-SCF/INCAR
1-PBE-SCF/KPOINTS
1-PBE-SCF/POSCAR
1-PBE-SCF/POTCAR
1-PBE-SCF/job.sh

sent 72.56K bytes  received 115 bytes  29.07K bytes/sec
total size is 196.69K  speedup is 2.71

sending incremental file list
2-PBE-DOS/
2-PBE-DOS/INCAR
2-PBE-DOS/KPOINTS
2-PBE-DOS/POSCAR
2-PBE-DOS/POTCAR
2-PBE-DOS/job.sh

sent 72.52K bytes  received 115 bytes  29.05K bytes/sec
total size is 196.67K  speedup is 2.71

sending incremental file list
3-PBE-BS/
3-PBE-BS/INCAR
3-PBE-BS/KPOINTS
3-PBE-BS/POSCAR
3-PBE-BS/POTCAR
3-PBE-BS/job.sh

sent 72.63K bytes  received 115 bytes  29.10K bytes/sec
total size is 197.07K  speedup is 2.71



Now that we created the Jobs and synced the files we can start the calculations. 
In this case we need to start the calculation of the first step, the others are automated.

In [8]:
job_first_step.run_job(sync=True) #sync files before running, in this case it wouldn't be necessary

sending incremental file list
POSCAR

sent 104 bytes  received 41 bytes  58.00 bytes/sec
total size is 161  speedup is 1.11

sending incremental file list
POTCAR

sent 106 bytes  received 1.72K bytes  728.40 bytes/sec
total size is 195.67K  speedup is 107.45

sending incremental file list
INCAR

sent 103 bytes  received 41 bytes  57.60 bytes/sec
total size is 226  speedup is 1.57

sending incremental file list
KPOINTS

sent 105 bytes  received 41 bytes  58.40 bytes/sec
total size is 80  speedup is 0.55

sending incremental file list
job.sh

sent 104 bytes  received 41 bytes  58.00 bytes/sec
total size is 545  speedup is 3.76

Submitted batch job 14746180
[I] Selected project: project01136
[I] Possible CPU types    = Haswell(avx2)
[I] Possible specials     = short



In [15]:
# First step should be finished, let's get the files. With the sync-hpc.py script running in the background 
#they will sync automatically periodically

job_first_step.sync_from_hpc()


receiving incremental file list
CHG
CHGCAR
CONTCAR
DOSCAR
EIGENVAL
IBZKPT
OSZICAR
OUTCAR
PCDAT
PROCAR
WAVECAR
XDATCAR
err.14746180
exit_status.txt
out.14746180
vasprun.xml

sent 328 bytes  received 6.14M bytes  945.03K bytes/sec
total size is 10.09M  speedup is 1.64



In [19]:
# now let's collect the outputs and analyse them
job_first_step.get_outputs()
job_first_step.is_converged

True

In [ ]:
ds.queue() 
# calculations are finished, we can sync them and collect outputs
ds.sync_jobs()

In [22]:
ds.get_jobs_outputs()

Now that we collected the calculations we can check the results

In [23]:
ds.jobs_table()

,formula,group,nodes,is_converged
job_name,,,,
Si-BS_PBE-el-str_1,Si2,1-PBE-SCF,,True
Si-BS_PBE-el-str_2,Si2,2-PBE-DOS,,True
Si-BS_PBE-el-str_3,Si2,3-PBE-BS,,True


In [24]:
# we could check also the total energies
ds.jobs_table(display=['final_energy'])

,formula,group,nodes,is_converged,final_energy
job_name,,,,,
Si-BS_PBE-el-str_1,Si2,1-PBE-SCF,,True,-10.840906
Si-BS_PBE-el-str_2,Si2,2-PBE-DOS,,True,-10.844797
Si-BS_PBE-el-str_3,Si2,3-PBE-BS,,True,-11.002882
